In [1]:
import pandas as pd

# from what?
# from Biosample table, linked to gold by gold sample IDs?
# specifically from NMDC

# also get reverse ? envo/mics proportial mappings to gold
gp_counts_fn = 'gold-path-mixs-triad-counts.xlsx'

gp_counts = pd.read_excel(gp_counts_fn)

In [2]:
gp_counts["gold_path_cat"] = (
    gp_counts.ECOSYSTEM
    + "_"
    + gp_counts.ECOSYSTEM_CATEGORY
    + "_"
    + gp_counts.ECOSYSTEM_TYPE
    + "_"
    + gp_counts.ECOSYSTEM_SUBTYPE
    + "_"
    + gp_counts.SPECIFIC_ECOSYSTEM
)

gp_counts.COUNT.fillna(0, inplace=True)
gp_counts.fillna('-', inplace=True)

gp_counts = gp_counts.astype(str)
gp_counts['COUNT'] = gp_counts['COUNT'].astype(float)

# gp_counts.BROAD_SCALE_LABEL.dropna('|')
# gp_counts.LOCAL_SCALE_LABEL.dropna('|')
# gp_counts.MEDIUM_LABEL.dropna('|')


gp_counts["mixs_cat"] = (
    gp_counts.BROAD_SCALE_LABEL
    + "_"
    + gp_counts.LOCAL_SCALE_LABEL
    + "_"
    + gp_counts.MEDIUM_LABEL
)

# gp_counts

In [3]:
gp_counts_min = gp_counts[['gold_path_cat','mixs_cat','COUNT']]
# gp_counts_min['COUNT'] = gp_counts_min['COUNT'].astype(float)
gp_counts_min

,gold_path_cat,mixs_cat,COUNT
0,Environmental_Aquatic_Marine_Unclassified_Uncl...,marine biome_marine water body_sea water,2754.0
1,Environmental_Aquatic_Marine_Oceanic_Unclassified,marine biome_marine water body_sea water,1496.0
2,Environmental_Air_Outdoor Air_Unclassified_Unc...,-_atmospheric layer_air,1117.0
3,Environmental_Aquatic_Marine_Inlet_Unclassified,marine biome_coastal inlet_sea water,1104.0
4,Environmental_Aquatic_Marine_Epipelagic_Unclas...,oceanic epipelagic zone biome_-_sea water,1016.0
...,...,...,...
1181,Environmental_Terrestrial_Volcanic_Fumaroles_U...,terrestrial biome_volcano_sediment,1.0
1182,Environmental_Terrestrial_Volcanic_Unclassifie...,terrestrial biome_mud volcano_sediment,1.0
1183,Environmental_Unclassified_Unclassified_Unclas...,-_-_-,1.0
1184,Environmental_Unclassified_Unclassified_Unclas...,terrestrial biome_fen_surface soil,1.0


In [4]:
summed_counts = gp_counts_min.groupby(['gold_path_cat']).sum()

In [5]:
# summed_counts['gold_path_cat'] = summed_counts.index
# summed_counts.columns = ['sum','gold_path_cat']
summed_counts.columns = ['sum']
summed_counts

,sum
gold_path_cat,
-,0.0
Environmental_Air_Indoor Air_Dust_Unclassified,122.0
Environmental_Air_Indoor Air_Unclassified_Unclassified,69.0
Environmental_Air_Outdoor Air_Unclassified_Unclassified,1147.0
Environmental_Aquatic_Aquaculture_Shrimp pond_Unclassified,48.0
...,...
Host-associated_Plants_Rhizosphere_Soil_Unclassified,56.0
Host-associated_Plants_Roots_Bulk soil_Unclassified,53.0
Host-associated_Plants_Roots_Rhizosphere_Soil,138.0


In [6]:
summed_counts.sort_values('sum', ascending=False)

,sum
gold_path_cat,
Environmental_Aquatic_Marine_Unclassified_Unclassified,3435.0
Environmental_Terrestrial_Soil_Unclassified_Unclassified,2531.0
Environmental_Aquatic_Marine_Oceanic_Unclassified,1901.0
Environmental_Terrestrial_Soil_Unclassified_Forest Soil,1549.0
Environmental_Aquatic_Freshwater_Lake_Unclassified,1493.0
...,...
Environmental_Aquatic_Marine_Mud volcano_Unclassified,1.0
Environmental_Aquatic_Marine_Intertidal zone_Oil-contaminated,1.0
Environmental_Aquatic_Freshwater_Ice_Ice accretions,1.0


In [7]:
gp_map_prop = gp_counts_min.merge(summed_counts, left_on='gold_path_cat', right_index=True)
gp_map_prop['proportion'] = gp_map_prop['COUNT'] / gp_map_prop['sum']


In [8]:
gp_map_prop.drop('COUNT', axis=1, inplace=True)

In [9]:
gp_counts_final = gp_counts.merge(
    gp_map_prop,
    left_on=["gold_path_cat", "mixs_cat"],
    right_on=["gold_path_cat", "mixs_cat"],
)



In [10]:
gp_counts_final.sort_values(['proportion', 'sum'], ascending=False, inplace=True)

In [11]:
gp_counts_final.to_csv('goldpaths_to_triads_by_proportion.tsv', sep='\t', index=False)

In [12]:
gp_counts_final

,ECOSYSTEM,ECOSYSTEM_CATEGORY,ECOSYSTEM_TYPE,ECOSYSTEM_SUBTYPE,SPECIFIC_ECOSYSTEM,BROAD_SCALE_LABEL,LOCAL_SCALE_LABEL,MEDIUM_LABEL,ENV_BROAD_SCALE,ENV_LOCAL_SCALE,ENV_MEDIUM,COUNT,gold_path_cat,mixs_cat,sum,proportion
4,Environmental,Aquatic,Marine,Epipelagic,Unclassified,oceanic epipelagic zone biome,-,sea water,ENVO_01000035,-,ENVO_00002149,1016.0,Environmental_Aquatic_Marine_Epipelagic_Unclas...,oceanic epipelagic zone biome_-_sea water,1016.0,1.000000
18,Environmental,Aquatic,Thermal springs,Hot (42-90C),Microbial mats,aquatic biome,hot spring,microbial mat,ENVO_00002030,ENVO_00000051,ENVO_01000008,243.0,Environmental_Aquatic_Thermal springs_Hot (42-...,aquatic biome_hot spring_microbial mat,243.0,1.000000
23,Environmental,Aquatic,Marine,Harbor,Unclassified,marine biome,harbor,sea water,ENVO_00000447,ENVO_00000463,ENVO_00002149,186.0,Environmental_Aquatic_Marine_Harbor_Unclassified,marine biome_harbor_sea water,186.0,1.000000
27,Environmental,Aquatic,Freshwater,Lake,Epilimnion,freshwater lake biome,epilimnion,lake water,ENVO_01000252,ENVO_00002131,ENVO_04000007,167.0,Environmental_Aquatic_Freshwater_Lake_Epilimnion,freshwater lake biome_epilimnion_lake water,167.0,1.000000
34,Environmental,Terrestrial,Floodplain,Soil,Unclassified,terrestrial biome,flood plain,soil,ENVO_00000446,ENVO_00000255,ENVO_00001998,144.0,Environmental_Terrestrial_Floodplain_Soil_Uncl...,terrestrial biome_flood plain_soil,144.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,Environmental,Aquatic,Marine,Unclassified,Unclassified,oceanic abyssopelagic zone biome,sea floor,deep marine sediment,ENVO_01000038,ENVO_00000482,ENVO_00002113,1.0,Environmental_Aquatic_Marine_Unclassified_Uncl...,oceanic abyssopelagic zone biome_sea floor_dee...,3435.0,0.000291
1034,Environmental,Aquatic,Marine,Unclassified,Unclassified,polar biome,glacial lake,meltwater,ENVO_01000339,ENVO_00000488,ENVO_01000722,1.0,Environmental_Aquatic_Marine_Unclassified_Uncl...,polar biome_glacial lake_meltwater,3435.0,0.000291
423,Environmental,Aquatic,Marine,Coastal,Sediment,-,-,-,-,-,-,0.0,Environmental_Aquatic_Marine_Coastal_Sediment,-_-_-,172.0,0.000000
425,Environmental,Aquatic,Marine,Coastal,Sediment,-,-,-,-,-,-,3.0,Environmental_Aquatic_Marine_Coastal_Sediment,-_-_-,172.0,0.000000
